In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy.sql import label

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite",echo=True)
this_conn = engine.connect()

2021-07-23 15:31:06,212 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-23 15:31:06,214 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:06,217 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-23 15:31:06,222 INFO sqlalchemy.engine.base.Engine ()


In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

2021-07-23 15:31:07,805 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-23 15:31:07,809 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:07,817 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("measurement")
2021-07-23 15:31:07,818 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:07,823 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2021-07-23 15:31:07,824 INFO sqlalchemy.engine.base.Engine ('measurement',)
2021-07-23 15:31:07,828 INFO sqlalchemy.engine.base.Engine PRAGMA main.foreign_key_list("measurement")
2021-07-23 15:31:07,830 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:07,832 INFO sqlalchemy.engine.base.Engine PRAGMA temp.foreign_key_list("measurement")
2021-07-23 15:31:07,833 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:07,835 INFO sqlalchemy.engine.base.Engine SELEC

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
### Create the inspector and connect it to the engine
inspector = inspect(engine)

In [8]:
# Save references to each table
### Collect the names of tables within the database
inspector.get_table_names()

2021-07-23 15:31:26,135 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2021-07-23 15:31:26,138 INFO sqlalchemy.engine.base.Engine ()


['measurement', 'station']

In [9]:
# get columns for measurement table
meas_cols = inspector.get_columns('measurement')
for c in meas_cols:
    print(c['name'], c['type'])

2021-07-23 15:31:28,611 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("measurement")
2021-07-23 15:31:28,612 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


In [10]:
# get columns for station table
stat_cols = inspector.get_columns('station')
for c in stat_cols:
    print(c['name'], c['type'])

2021-07-23 15:31:32,663 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_xinfo("station")
2021-07-23 15:31:32,666 INFO sqlalchemy.engine.base.Engine ()
id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [11]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
measurement_data = engine.execute("SELECT * FROM measurement")
measurement_df = pd.read_sql("SELECT * FROM measurement", this_conn)

measurement_df.head()

2021-07-23 15:31:39,566 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-23 15:31:39,568 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:39,571 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM measurement")
2021-07-23 15:31:39,573 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:39,575 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM measurement")
2021-07-23 15:31:39,579 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:39,580 INFO sqlalchemy.engine.base.Engine SELECT * FROM measurement
2021-07-23 15:31:39,582 INFO sqlalchemy.engine.base.Engine ()


,id,station,date,prcp,tobs
0,1,USC00519397,2010-01-01,0.08,65.0
1,2,USC00519397,2010-01-02,0.00,63.0
2,3,USC00519397,2010-01-03,0.00,74.0
3,4,USC00519397,2010-01-04,0.00,76.0
4,5,USC00519397,2010-01-06,NaN,73.0


In [12]:
# not sure I need these, but doing it because it was in very first activity Query All Records in the Database
station_data = engine.execute("SELECT * FROM station")
station_df = pd.read_sql("SELECT * FROM station", this_conn)

station_df

2021-07-23 15:31:45,785 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-23 15:31:45,787 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:45,790 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("SELECT * FROM station")
2021-07-23 15:31:45,791 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:45,794 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("SELECT * FROM station")
2021-07-23 15:31:45,795 INFO sqlalchemy.engine.base.Engine ()
2021-07-23 15:31:45,797 INFO sqlalchemy.engine.base.Engine SELECT * FROM station
2021-07-23 15:31:45,798 INFO sqlalchemy.engine.base.Engine ()


,id,station,name,latitude,longitude,elevation
0,1,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,2,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,3,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,4,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,5,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,6,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,7,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,8,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,9,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [13]:
### Assign classes to variables 
Measurement = Base.classes.measurement
Station = Base.classes.station

In [14]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [15]:
### This is the first date as a check to make sure things were working while I searched for last date
# Find the most recent date in the data set.
earliest_date = session.query(Measurement.date).order_by(Measurement.date).first()
print(earliest_date)

2021-07-23 15:31:55,787 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-07-23 15:31:55,789 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date 
FROM measurement ORDER BY measurement.date
 LIMIT ? OFFSET ?
2021-07-23 15:31:55,791 INFO sqlalchemy.engine.base.Engine (1, 0)
('2010-01-01',)


In [16]:
# find most recent date in the database
last_date = session.query(func.max(Measurement.date)).first()
print(last_date)

2021-07-23 15:31:59,088 INFO sqlalchemy.engine.base.Engine SELECT max(measurement.date) AS max_1 
FROM measurement
 LIMIT ? OFFSET ?
2021-07-23 15:31:59,089 INFO sqlalchemy.engine.base.Engine (1, 0)
('2017-08-23',)


## Design a query to retrieve the last 12 months of precipitation data and plot the results. 

In [17]:
# Starting from the most recent data point in the database. 
# The date that was pulled was a tuple & I saw online to use a marker to pull just the date 
# end_date = dt.date(last_date[0])  # The date is 2017-08-23; *the code DOES WORK, but I hard-coded to be sure*
end_hand = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
# start_date = end_date - dt.timedelta(days=365)  # THIS date should be 2016-08-24; hadn't gotten to work yet
start_hand = dt.date(2016, 8, 24) 
print(f'start date is {start_hand} & end date is {end_hand}')

start date is 2016-08-24 & end date is 2017-08-23


# Starting from the most recent data point in the database. 
# The date that was pulled was a tuple & I saw online to use a marker to pull just the date 
end_date = dt.date(last_date[0])  # The date is 2017-08-23; ***the code DID WORK***
# print(end_date)
# end_hand = dt.date(2017, 8, 23)
# Calculate the date one year from the last date in data set.
start_date = end_date - dt.timedelta(days=365)  # THIS date should be 2016-08-24; hadn't gotten to work yet
# start_hand = dt.date(2016, 8, 24) 
print(f'start date is {start_date} & end date is {end_date}')

sel = [Dow.stock, 
       func.avg(Dow.open_price), 
       func.avg(Dow.high_price), 
       func.avg(Dow.low_price), 
       func.avg(Dow.close_price)]
may_averages = session.query(*sel).\
    filter(func.strftime("%m", Dow.date) == "05").\
    group_by(Dow.stock).\
    order_by(Dow.stock).all()
may_averages

In [46]:
# V0 produces list of tuples; has appropriate dates, but need to collapse so there is only 1 pair per date
precip_output0 = session.query(Measurement.id, Measurement.date, func.sum(Measurement.prcp)).\
    filter(Measurement.date >=start_hand).\
    group_by(Measurement.date).all()
    # order_by(Measurement.date)
    # all()

2021-07-23 15:59:11,348 INFO sqlalchemy.engine.base.Engine SELECT measurement.id AS measurement_id, measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE measurement.date >= ? GROUP BY measurement.date
2021-07-23 15:59:11,351 INFO sqlalchemy.engine.base.Engine ('2016-08-24',)


In [47]:
print(precip_output0)

[(2365, '2016-08-24', 9.329999999999998), (2366, '2016-08-25', 0.54), (2367, '2016-08-26', 0.09999999999999999), (2368, '2016-08-27', 0.31999999999999995), (2369, '2016-08-28', 3.0999999999999996), (2370, '2016-08-29', 1.46), (2371, '2016-08-30', 0.07), (2372, '2016-08-31', 3.1799999999999997), (2373, '2016-09-01', 0.03), (2374, '2016-09-02', 0.25), (2375, '2016-09-03', 1.27), (2376, '2016-09-04', 1.3800000000000001), (2377, '2016-09-05', 0.33999999999999997), (2378, '2016-09-06', 1.23), (2379, '2016-09-07', 2.0), (2380, '2016-09-08', 0.4600000000000001), (2381, '2016-09-09', 1.0699999999999998), (2382, '2016-09-10', 1.4), (2383, '2016-09-11', 1.25), (2384, '2016-09-12', 1.54), (2385, '2016-09-13', 2.71), (2386, '2016-09-14', 14.280000000000001), (2387, '2016-09-15', 4.96), (2388, '2016-09-16', 0.8200000000000001), (2389, '2016-09-17', 0.6799999999999999), (2390, '2016-09-18', 0.53), (2391, '2016-09-19', 0.32), (2392, '2016-09-20', 1.03), (2393, '2016-09-21', 1.17), (2394, '2016-09-22'

In [171]:
# V1 * * * This version creates empty list
# INSTRUCTIONS: Perform a query to retrieve the data and precipitation scores
# So what you are actually asking for is a sum across all of the stations? 
# As opposed to plotting each station separately?

## currently has HARD-CODED date

precip_output1 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date) >= start_hand).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all() # when I take out .all(), it doesn't seem to do anything

2021-07-23 13:42:11,228 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date) >= ? GROUP BY measurement.date ORDER BY measurement.date
2021-07-23 13:42:11,230 INFO sqlalchemy.engine.base.Engine ('%m', '2016-08-24')


In [ ]:
# V2 * * * This version creates a list of tuples, BUT has ALL DATES; 
# difference btwn V1 & V2 is where parens are in filter line

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output2 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all()
    # group_by(Measurement.date) # returns only the first station's measurements -- WHY?!?!

2021-07-23 13:42:33,389 INFO sqlalchemy.engine.base.Engine SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?) GROUP BY measurement.date ORDER BY measurement.date
2021-07-23 13:42:33,392 INFO sqlalchemy.engine.base.Engine ('%m', '2016-08-24')


In [175]:
# V3 * * * This version doesn't even seem to populate the filter

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output3 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)).\
    order_by(Measurement.date)#.all()


In [178]:
# V4 * * * This version doesn't even seem to populate the filter

## currently has HARD-CODED date
#precip_output = session.query(Measurement.date, Measurement.station, Measurement.prcp).\

precip_output4 = session.query(Measurement.date, func.sum(Measurement.prcp)).\
    filter(func.strftime("%m", Measurement.date >= start_hand)) #.\
    # order_by(Measurement.date)#.all()


In [172]:
# It's not running the filter when I put .all(), but when I don't put .all(), it won't show me anything
print(precip_output1)

[]


In [174]:
print(precip_output2)

[('2010-01-01', 1.06), ('2010-01-02', 0.02), ('2010-01-03', 0.0), ('2010-01-04', 0.01), ('2010-01-05', 0.03), ('2010-01-06', 0.5800000000000001), ('2010-01-07', 1.65), ('2010-01-08', 0.06), ('2010-01-09', 0.02), ('2010-01-10', 0.02), ('2010-01-11', 0.8800000000000001), ('2010-01-12', 0.02), ('2010-01-13', 0.02), ('2010-01-14', 0.0), ('2010-01-15', 0.0), ('2010-01-16', 0.0), ('2010-01-17', 0.0), ('2010-01-18', 0.0), ('2010-01-19', 0.0), ('2010-01-20', 0.01), ('2010-01-21', 0.65), ('2010-01-22', 0.0), ('2010-01-23', 0.0), ('2010-01-24', 0.02), ('2010-01-25', 0.0), ('2010-01-26', 0.25), ('2010-01-27', 1.6), ('2010-01-28', 0.73), ('2010-01-29', 2.19), ('2010-01-30', 7.3500000000000005), ('2010-01-31', 2.16), ('2010-02-01', 0.05), ('2010-02-02', 0.05), ('2010-02-03', 3.21), ('2010-02-04', 0.02), ('2010-02-05', 0.16), ('2010-02-06', 0.12), ('2010-02-07', 0.01), ('2010-02-08', 0.0), ('2010-02-09', 0.0), ('2010-02-10', 0.0), ('2010-02-11', 0.0), ('2010-02-12', 0.8200000000000001), ('2010-02-13

In [176]:
print(precip_output3)

SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?) ORDER BY measurement.date


In [179]:
print(precip_output4)

SELECT measurement.date AS measurement_date, sum(measurement.prcp) AS sum_1 
FROM measurement 
WHERE strftime(?, measurement.date >= ?)


In [33]:
print(precip_output0)

[(2365, '2016-08-24', 0.08), (2366, '2016-08-25', 0.08), (2367, '2016-08-26', 0.0), (2368, '2016-08-27', 0.0), (2369, '2016-08-28', 0.01), (2370, '2016-08-29', 0.0), (2371, '2016-08-30', 0.0), (2372, '2016-08-31', 0.13), (2373, '2016-09-01', 0.0), (2374, '2016-09-02', 0.0), (2375, '2016-09-03', 0.0), (2376, '2016-09-04', 0.03), (2377, '2016-09-05', None), (2378, '2016-09-06', None), (2379, '2016-09-07', 0.05), (2380, '2016-09-08', 0.0), (2381, '2016-09-09', 0.03), (2382, '2016-09-10', 0.0), (2383, '2016-09-11', 0.05), (2384, '2016-09-12', 0.0), (2385, '2016-09-13', 0.02), (2386, '2016-09-14', 1.32), (2387, '2016-09-15', 0.42), (2388, '2016-09-16', 0.06), (2389, '2016-09-17', 0.05), (2390, '2016-09-18', 0.0), (2391, '2016-09-19', 0.0), (2392, '2016-09-20', 0.0), (2393, '2016-09-21', 0.0), (2394, '2016-09-22', 0.02), (2395, '2016-09-23', 0.0), (2396, '2016-09-24', 0.0), (2397, '2016-09-25', 0.0), (2398, '2016-09-26', 0.06), (2399, '2016-09-27', 0.02), (2400, '2016-09-28', 0.0), (2401, '2

In [48]:
# Save the query results as a Pandas DataFrame and set the index to the date column 
# -- which means precipitation is summed on date above
# **trying to pass list of tuples** 7-22; 7-23 now an empty list
precipitation_df = pd.DataFrame(precip_output0, columns=['id', 'date', 'precip']).set_index('id')
precipitation_df.dtypes

date       object
precip    float64
dtype: object

In [49]:
# Sort the dataframe by date
precipitation_df['date']=pd.to_datetime(precipitation_df['date'])
precipitation_df = precipitation_df.sort_values(by='date')
precipitation_df

,date,precip
id,,
2365,2016-08-24,9.33
2366,2016-08-25,0.54
2367,2016-08-26,0.10
2368,2016-08-27,0.32
2369,2016-08-28,3.10
...,...,...
2720,2017-08-19,0.09
2721,2017-08-20,0.01
2722,2017-08-21,0.58


In [ ]:
## Use Pandas Plotting with Matplotlib to plot the data


In [ ]:
## Use Pandas to calcualte the summary statistics for the precipitation data
precip_summary_df = precipitation_df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset


In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram


# Close session

In [ ]:
# Close Session
session.close()